Fully Personalized

In [1]:
import pytz
import os


DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul

PATH_DATA = 'data/D'
PATH_ESM = os.path.join(PATH_DATA, 'EsmResponse.csv')
PATH_PARTICIPANT = os.path.join(PATH_DATA, 'UserInfo.csv')
PATH_SENSOR = os.path.join(PATH_DATA, 'Sensor')

PATH_INTERMEDIATE = os.path.join('data/intermediate')

DATA_TYPES = {
    'Acceleration': 'ACC',
    'AmbientLight': 'AML',
    'Calorie': 'CAL',
    'Distance': 'DST',
    'EDA': 'EDA',
    'HR': 'HRT',
    'RRI': 'RRI',
    'SkinTemperature': 'SKT',
    'StepCount': 'STP',
    'UltraViolet': 'ULV',
    'ActivityEvent': 'ACE',
    'ActivityTransition': 'ACT',
    'AppUsageEvent': 'APP',
    'BatteryEvent': 'BAT',
    'CallEvent': 'CAE',
    'Connectivity': 'CON',
    'DataTraffic': 'DAT',
    'InstalledApp': 'INS',
    'Location': 'LOC',
    'MediaEvent': 'MED',
    'MessageEvent': 'MSG',
    'WiFi': 'WIF',
    'ScreenEvent': 'SCR',
    'RingerModeEvent': 'RNG',
    'ChargeEvent': 'CHG',
    'PowerSaveEvent': 'PWS',
    'OnOffEvent': 'ONF'
}

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import cloudpickle
import ray
from datetime import datetime
from contextlib import contextmanager
import warnings
import time


def load(path: str):
    with open(path, mode='rb') as f:
        return cloudpickle.load(f)

    
def dump(obj, path: str):
    with open(path, mode='wb') as f:
        cloudpickle.dump(obj, f)
        
    
def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))


def summary(x):
    x = np.asarray(x)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        n = len(x)
        # Here, uppercase np.dtype.kind corresponds to non-numeric data.
        # Also, we view the boolean data as dichotomous categorical data.
        if x.dtype.kind.isupper() or x.dtype.kind == 'b': 
            cnt = pd.Series(x).value_counts(dropna=False)
            card = len(cnt)
            cnt = cnt[:20]                
            cnt_str = ', '.join([f'{u}:{c}' for u, c in zip(cnt.index, cnt)])
            if card > 30:
                cnt_str = f'{cnt_str}, ...'
            return {
                'n': n,
                'cardinality': card,
                'value_count': cnt_str
            }
        else: 
            x_nan = x[np.isnan(x)]
            x_norm = x[~np.isnan(x)]
            
            tot = np.sum(x_norm)
            m = np.mean(x_norm)
            me = np.median(x_norm)
            s = np.std(x_norm, ddof=1)
            l, u = np.min(x_norm), np.max(x)
            conf_l, conf_u = st.t.interval(0.95, len(x_norm) - 1, loc=m, scale=st.sem(x_norm))
            n_nan = len(x_nan)
            
            return {
                'n': n,
                'sum': tot,
                'mean': m,
                'SD': s,
                'med': me,
                'range': (l, u),
                'conf.': (conf_l, conf_u),
                'nan_count': n_nan
            }


@contextmanager
def on_ray(*args, **kwargs):
    try:
        if ray.is_initialized():
            ray.shutdown()
        ray.init(*args, **kwargs)
        yield None
    finally:
        ray.shutdown()

transform = {
    'GAME': 'ENTER',
    'GAME_TRIVIA': 'ENTER',
    'GAME_CASINO': 'ENTER',
    'GAME-ACTION': 'ENTER',
    'GAME_SPORTS': 'ENTER',
    'GAME_PUZZLE': 'ENTER',
    'GAME_SIMULATION': 'ENTER',
    'GAME_STRATEGY': 'ENTER',
    'GAME_ROLE_PLAYING': 'ENTER',
    'GAME_ACTION': 'ENTER',
    'GAME_ARCADE': 'ENTER',
    'GAME_RACING': 'ENTER',
    'GAME_CASUAL': 'ENTER',
    'GAME_MUSIC': 'ENTER',
    'GAME_CARD': 'ENTER',
    'GAME_ADVENTURE': 'ENTER',
    'GAME_BOARD': 'ENTER',
    'GAME_EDUCATIONAL': 'ENTER',
    'GAME_RACING': 'ENTER',
    'PHOTOGRAPHY': 'ENTER',
    'ENTERTAINMENT': 'ENTER',
    'SPORTS': 'ENTER',
    'MUSIC_AND_AUDIO': 'ENTER',
    'COMICS': 'ENTER',
    'VIDEO_PLAYERS_AND_EDITORS': 'ENTER',
    'VIDEO_PLAYERS': 'ENTER',
    'ART_AND_DESIGN': 'ENTER',
    'TRAVEL_AND_LOCAL': 'INFO',
    'FOOD_AND_DRINK': 'INFO',
    'NEWS_AND_MAGAZINES': 'INFO',
    'MAPS_AND_NAVIGATION': 'INFO',
    'WEATHER': 'INFO',
    'HOUSE_AND_HOME': 'INFO',
    'BOOKS_AND_REFERENCE': 'INFO',
    'SHOPPING': 'INFO',
    'LIBRARIES_AND_DEMO': 'INFO',
    'BEAUTY': 'INFO',
    'AUTO_AND_VEHICLES': 'INFO',
    'LIFESTYLE': 'INFO',
    'PERSONALIZATION': 'SYSTEM',
    'TOOLS': 'SYSTEM',
    'COMMUNICATION': 'SOCIAL',
    'SOCIAL': 'SOCIAL',
    'DATING': 'SOCIAL',
    'PARENTING':'SOCIAL',
    'FINANCE': 'WORK',
    'BUSINESS': 'WORK',
    'PRODUCTIVITY': 'WORK',
    'EDUCATION': 'WORK',
    'HEALTH_AND_FITNESS': 'HEALTH',
    'MEDICAL': 'HEALTH',
    'SYSTEM': 'SYSTEM',
    'MISC': 'SYSTEM', # ABC logger
     None: 'UNKNOWN',
    'UNKNOWN':'UNKNOWN'
}

In [3]:
p = os.path.join(PATH_INTERMEDIATE, 'stress_baseline.pkl')
X, y, groups, t, datetimes = load(p)

In [4]:
similar_user = pd.read_csv(os.path.join(PATH_INTERMEDIATE,  'similar_user_5.csv'))
similar_user

,pcode,BFI_OPN,BFI_CON,BFI_NEU,BFI_EXT,BFI_AGR,cluster
0,P01,11,11,3,4,13,2
1,P02,14,5,12,14,5,3
2,P03,10,15,8,7,11,4
3,P05,10,11,13,10,6,3
4,P06,3,6,11,3,6,1
5,P08,10,8,9,9,12,0
6,P09,12,12,4,11,9,3
7,P10,6,7,9,9,11,0
8,P12,9,12,7,7,12,4
9,P13,5,12,3,12,13,4


In [5]:
feat_current = X.loc[:,[('#VAL' in str(x)) or ('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_dsc = X.loc[:,[('#DSC' in str(x))  for x in X.keys()]]  
feat_yesterday = X.loc[:,[('Yesterday' in str(x))  for x in X.keys()]]  
feat_today = X.loc[:,[('Today' in str(x))  for x in X.keys()]]  
feat_sleep = X.loc[:,[('Sleep' in str(x))  for x in X.keys()]]  
feat_time = X.loc[:,[('Time' in str(x))  for x in X.keys()]]  
feat_pif = X.loc[:,[('PIF' in str(x))  for x in X.keys()]]  
feat_ImmediatePast = X.loc[:,[('ImmediatePast' in str(x))  for x in X.keys()]]
#Divide the time window features into sensor/past stress label
feat_current_sensor = X.loc[:,[('#VAL' in str(x))  for x in X.keys()]]  
feat_current_ESM = X.loc[:,[('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_ImmediatePast_sensor = feat_ImmediatePast.loc[:,[('ESM' not in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_ImmediatePast_ESM = feat_ImmediatePast.loc[:,[('ESM'  in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_today_sensor = feat_today.loc[:,[('ESM' not in str(x)) for x in feat_today.keys()]]  
feat_today_ESM = feat_today.loc[:,[('ESM'  in str(x)) for x in feat_today.keys()]]  
feat_yesterday_sensor = feat_yesterday.loc[:,[('ESM' not in str(x)) for x in feat_yesterday.keys()]]  
feat_yesterday_ESM = feat_yesterday.loc[:,[('ESM'  in str(x)) for x in feat_yesterday.keys()]] 
#Prepare the final feature set
feat_baseline = pd.concat([ feat_time,feat_dsc,feat_current_sensor, feat_ImmediatePast_sensor],axis=1)
feat_final = pd.concat([feat_baseline, feat_today_sensor],axis=1)
X = feat_final

In [6]:
import pandas as pd

# Convert groups to a pandas DataFrame
groups_df = pd.DataFrame(groups, columns=['pcode'])

# Ensure the data types of the 'pcode' and 'pcode' columns are the same
groups_df['pcode'] = groups_df['pcode'].astype(str)
similar_user['pcode'] = similar_user['pcode'].astype(str)

# Create a new Series that maps Pcode to cluster label
cluster_map = similar_user.set_index('pcode')['cluster']

# Create the 'cluster' column in the 'groups' DataFrame
groups_df['cluster'] = groups_df['pcode'].map(cluster_map)

In [7]:
print(groups_df)

     pcode  cluster
0      P01        2
1      P01        2
2      P01        2
3      P01        2
4      P01        2
...    ...      ...
2614   P80        2
2615   P80        2
2616   P80        2
2617   P80        2
2618   P80        2

[2619 rows x 2 columns]


In [8]:
# Add the 'cluster' column from 'groups_df' to 'X' as the first column
X.insert(0, 'cluster', groups_df['cluster'])

In [9]:
X

,cluster,Time#DOW=MON,Time#DOW=TUE,Time#DOW=WED,Time#DOW=THU,Time#DOW=FRI,Time#DOW=SAT,Time#DOW=SUN,Time#WKD=Y,Time#WKD=N,...,MED_VID#MEDTodayEvening,MED_VID#TSCTodayEvening,MED_VID#AVGTodayNight,MED_VID#STDTodayNight,MED_VID#SKWTodayNight,MED_VID#KURTodayNight,MED_VID#ASCTodayNight,MED_VID#BEPTodayNight,MED_VID#MEDTodayNight,MED_VID#TSCTodayNight
0,2,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2614,2,False,False,False,False,False,False,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2615,2,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2616,2,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2617,2,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
y

array([1, 1, 0, ..., 1, 0, 1])

In [11]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is df and the numpy array is y

# Get unique clusters
clusters = X['cluster'].unique()

# Create a dictionary to store each cluster's DataFrame
cluster_dfs = {}
cluster_labels = {}
cluster_groups = {}

for cluster in clusters:
    # Get the data for the current cluster
    cluster_dfs[cluster] = X[X['cluster'] == cluster]
    
    # Get the corresponding labels for the current cluster
    cluster_labels[cluster] = y[X['cluster'] == cluster]
    
    cluster_groups[cluster] = groups_df[X['cluster'] == cluster]

# Now you have separated DataFrames and numpy arrays for each cluster
# You can access them by:
# cluster_dfs[cluster_number]
# cluster_arrays[cluster_number]

In [12]:
cluster_dfs[0]

,cluster,Time#DOW=MON,Time#DOW=TUE,Time#DOW=WED,Time#DOW=THU,Time#DOW=FRI,Time#DOW=SAT,Time#DOW=SUN,Time#WKD=Y,Time#WKD=N,...,MED_VID#MEDTodayEvening,MED_VID#TSCTodayEvening,MED_VID#AVGTodayNight,MED_VID#STDTodayNight,MED_VID#SKWTodayNight,MED_VID#KURTodayNight,MED_VID#ASCTodayNight,MED_VID#BEPTodayNight,MED_VID#MEDTodayNight,MED_VID#TSCTodayNight
235,0,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236,0,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
237,0,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,-3.0,-3.0,0.0,0.0,0.0,0.0
238,0,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,-3.0,-3.0,0.0,0.0,0.0,0.0
239,0,False,False,True,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,-3.0,-3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,0,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2568,0,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2569,0,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2570,0,True,False,False,False,False,False,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
cluster_groups[0]

,pcode,cluster
235,P08,0
236,P08,0
237,P08,0
238,P08,0
239,P08,0
...,...,...
2567,P79,0
2568,P79,0
2569,P79,0
2570,P79,0


In [14]:
clusters = similar_user['cluster'].unique()

cluster_similarity = {}

for cluster in clusters:
    # Get the data for the current cluster
    cluster_similarity[cluster] = similar_user[similar_user['cluster'] == cluster]

In [15]:
cluster_similarity[0]

,pcode,BFI_OPN,BFI_CON,BFI_NEU,BFI_EXT,BFI_AGR,cluster
5,P08,10,8,9,9,12,0
7,P10,6,7,9,9,11,0
15,P28,6,9,8,7,11,0
16,P30,7,9,14,9,14,0
17,P31,9,10,9,9,10,0
23,P42,7,10,9,13,10,0
25,P47,8,12,11,7,12,0
26,P48,11,11,10,8,11,0
29,P51,6,9,9,4,13,0
36,P66,8,10,8,8,10,0


In [16]:
random_element = cluster_similarity[0].sample().iloc[0]
print(random_element)

pcode      P47
BFI_OPN      8
BFI_CON     12
BFI_NEU     11
BFI_EXT      7
BFI_AGR     12
cluster      0
Name: 25, dtype: object


In [17]:
def process_dataframe(X):
    """
    Process the input DataFrame 'X':
    1. Set column names as a range from 0 to the number of columns.
    2. Replace boolean values with 1 for True and 0 for False.

    Parameters:
        X (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The processed DataFrame with column names as numbers.
    """
    # Set column names as a range from 0 to the number of columns
    X.columns = range(X.shape[1])

    # Replace boolean values with 1 for True and 0 for False
    X = X * 1

    return X

In [18]:
import os
import pandas as pd
import numpy as np

def split_train_test(df, labels, indices):
    test_X = df.loc[indices]
    test_y = labels.loc[indices]
    train_X = df.drop(indices)
    train_y = labels.drop(indices)
    return train_X, train_y, test_X, test_y

In [19]:
import xgboost as xgb
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

auc_scores = []
f1_scores = []

params = {
    'max_depth': 6,
    'eta': 0.3,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    "gamma": 0,
    "subsample": 1,
    "colsample_bytree": 1,
    "colsample_bylevel": 1,
    "scale_pos_weight": 1,
    "reg_alpha": 0,
    "reg_lambda": 1,
}

In [20]:
random_seed = 0
for turn in range(5):
    random_seed = random_seed + 25
    for y in range(5):
        # Select 1 person from cluster_similarity(y) randomly
        random_element = cluster_similarity[y].sample(random_state=random_seed).iloc[0]
        random_seed = random_seed + 100
        matching_indices = []
        for index, element in cluster_groups[y].iterrows():
            if element['pcode'] == random_element['pcode']:
                matching_indices.append(index)
        # convert cluster_labels[y] into DataFrame with matching indices to cluster_dfs[y]
        labels_df = pd.DataFrame(cluster_labels[y], index=cluster_dfs[y].index)
        train_X, train_y, test_X, test_y = split_train_test(cluster_dfs[y], 
                                                            labels_df,
                                                            matching_indices)
        test_X = process_dataframe(test_X)
        train_X = process_dataframe(train_X)
        # Drop the first column for ST_XGboost
        train_X = train_X.iloc[:, 1:]
        test_X = test_X.iloc[:, 1:]
        
        dtrain = xgb.DMatrix(train_X, label=train_y.values)
        dtest = xgb.DMatrix(test_X, label=test_y.values)
        deval = xgb.DMatrix(test_X, label=test_y.values)

        # train
        evallist = [(dtrain, 'train'), (deval, 'eval')]
        bst = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dtest, 'eval')], early_stopping_rounds=200)
        # predict probabilities
        y_pred_proba = bst.predict(dtest)
        # Convert predicted probabilities to class labels
        y_pred_labels = (y_pred_proba > 0.5).astype(int)

        y_real = dtest.get_label()

        # Calculate AUC
        auc = roc_auc_score(y_real, y_pred_proba)
        print(auc)
        auc_scores.append(auc)

        # Calculate F1 score
        f1 = f1_score(y_real, y_pred_labels, average='macro')
        f1_scores.append(f1)

        
        
# print("labels")
# print(clusters_test_y[0])
# print("data")
# print(clusters_test_X[0])

[0]	eval-auc:0.62963
[1]	eval-auc:0.59407
[2]	eval-auc:0.52074
[3]	eval-auc:0.48000
[4]	eval-auc:0.50074
[5]	eval-auc:0.50741
[6]	eval-auc:0.50370
[7]	eval-auc:0.51704
[8]	eval-auc:0.52444
[9]	eval-auc:0.55556
[10]	eval-auc:0.55259
[11]	eval-auc:0.51111
[12]	eval-auc:0.52296
[13]	eval-auc:0.54222
[14]	eval-auc:0.52741
[15]	eval-auc:0.53926
[16]	eval-auc:0.54815
[17]	eval-auc:0.55704
[18]	eval-auc:0.57333
[19]	eval-auc:0.57185
[20]	eval-auc:0.56444
[21]	eval-auc:0.57481
[22]	eval-auc:0.55556
[23]	eval-auc:0.54222
[24]	eval-auc:0.52444
[25]	eval-auc:0.54074
[26]	eval-auc:0.54222
[27]	eval-auc:0.54667
[28]	eval-auc:0.54370
[29]	eval-auc:0.55407
[30]	eval-auc:0.55852
[31]	eval-auc:0.56296
[32]	eval-auc:0.56296
[33]	eval-auc:0.56593
[34]	eval-auc:0.56444
[35]	eval-auc:0.56148
[36]	eval-auc:0.55704
[37]	eval-auc:0.56593
[38]	eval-auc:0.56444
[39]	eval-auc:0.56000
[40]	eval-auc:0.57037
[41]	eval-auc:0.58074
[42]	eval-auc:0.57630
[43]	eval-auc:0.57630
[44]	eval-auc:0.56889
[45]	eval-auc:0.5688

[164]	eval-auc:0.34490
[165]	eval-auc:0.34490
[166]	eval-auc:0.34490
[167]	eval-auc:0.34490
[168]	eval-auc:0.34490
[169]	eval-auc:0.34490
[170]	eval-auc:0.34286
[171]	eval-auc:0.34286
[172]	eval-auc:0.34082
[173]	eval-auc:0.34286
[174]	eval-auc:0.34082
[175]	eval-auc:0.34286
[176]	eval-auc:0.34286
[177]	eval-auc:0.34082
[178]	eval-auc:0.33673
[179]	eval-auc:0.34286
[180]	eval-auc:0.33673
[181]	eval-auc:0.33673
[182]	eval-auc:0.33469
[183]	eval-auc:0.33673
[184]	eval-auc:0.33673
[185]	eval-auc:0.33673
[186]	eval-auc:0.33673
[187]	eval-auc:0.33673
[188]	eval-auc:0.33673
[189]	eval-auc:0.33673
[190]	eval-auc:0.33673
[191]	eval-auc:0.33469
[192]	eval-auc:0.33673
[193]	eval-auc:0.33878
[194]	eval-auc:0.33878
[195]	eval-auc:0.33469
[196]	eval-auc:0.33265
[197]	eval-auc:0.33469
[198]	eval-auc:0.33469
[199]	eval-auc:0.33469
0.3346938775510204
[0]	eval-auc:0.38221
[1]	eval-auc:0.36779
[2]	eval-auc:0.38542
[3]	eval-auc:0.40224
[4]	eval-auc:0.36058
[5]	eval-auc:0.37821
[6]	eval-auc:0.38141
[7]	ev

[325]	eval-auc:0.42147
[326]	eval-auc:0.42147
[327]	eval-auc:0.42147
[328]	eval-auc:0.42147
[329]	eval-auc:0.42147
[330]	eval-auc:0.42147
[331]	eval-auc:0.42147
[332]	eval-auc:0.42147
[333]	eval-auc:0.42147
[334]	eval-auc:0.42147
[335]	eval-auc:0.42147
[336]	eval-auc:0.42147
[337]	eval-auc:0.42147
[338]	eval-auc:0.42147
[339]	eval-auc:0.42147
[340]	eval-auc:0.42147
[341]	eval-auc:0.42147
[342]	eval-auc:0.42147
[343]	eval-auc:0.42147
[344]	eval-auc:0.42147
[345]	eval-auc:0.42147
[346]	eval-auc:0.42147
[347]	eval-auc:0.42147
[348]	eval-auc:0.42147
[349]	eval-auc:0.42147
[350]	eval-auc:0.42147
[351]	eval-auc:0.42147
[352]	eval-auc:0.42147
[353]	eval-auc:0.42147
[354]	eval-auc:0.42147
[355]	eval-auc:0.42147
[356]	eval-auc:0.42147
[357]	eval-auc:0.42147
[358]	eval-auc:0.42147
[359]	eval-auc:0.42147
[360]	eval-auc:0.42147
[361]	eval-auc:0.42147
[362]	eval-auc:0.42147
[363]	eval-auc:0.42147
[364]	eval-auc:0.42147
[365]	eval-auc:0.42147
[366]	eval-auc:0.42147
[367]	eval-auc:0.42147
[368]	eval-

[33]	eval-auc:0.51852
[34]	eval-auc:0.54938
[35]	eval-auc:0.55556
[36]	eval-auc:0.55556
[37]	eval-auc:0.56173
[38]	eval-auc:0.54938
[39]	eval-auc:0.56173
[40]	eval-auc:0.57407
[41]	eval-auc:0.55556
[42]	eval-auc:0.56173
[43]	eval-auc:0.56173
[44]	eval-auc:0.56173
[45]	eval-auc:0.56790
[46]	eval-auc:0.56790
[47]	eval-auc:0.57407
[48]	eval-auc:0.57407
[49]	eval-auc:0.58025
[50]	eval-auc:0.58025
[51]	eval-auc:0.57407
[52]	eval-auc:0.58025
[53]	eval-auc:0.55556
[54]	eval-auc:0.55556
[55]	eval-auc:0.55556
[56]	eval-auc:0.55556
[57]	eval-auc:0.55556
[58]	eval-auc:0.54321
[59]	eval-auc:0.54938
[60]	eval-auc:0.56173
[61]	eval-auc:0.55556
[62]	eval-auc:0.56173
[63]	eval-auc:0.56173
[64]	eval-auc:0.57407
[65]	eval-auc:0.58025
[66]	eval-auc:0.58642
[67]	eval-auc:0.58642
[68]	eval-auc:0.59259
[69]	eval-auc:0.59259
[70]	eval-auc:0.58642
[71]	eval-auc:0.58642
[72]	eval-auc:0.58642
[73]	eval-auc:0.58642
[74]	eval-auc:0.58642
[75]	eval-auc:0.58642
[76]	eval-auc:0.58642
[77]	eval-auc:0.59259
[78]	eval-

[105]	eval-auc:0.65857
[106]	eval-auc:0.65722
[107]	eval-auc:0.65722
[108]	eval-auc:0.65857
[109]	eval-auc:0.65992
[110]	eval-auc:0.66262
[111]	eval-auc:0.65857
[112]	eval-auc:0.65857
[113]	eval-auc:0.65587
[114]	eval-auc:0.65722
[115]	eval-auc:0.66532
[116]	eval-auc:0.66397
[117]	eval-auc:0.66262
[118]	eval-auc:0.65992
[119]	eval-auc:0.65452
[120]	eval-auc:0.65452
[121]	eval-auc:0.65722
[122]	eval-auc:0.65857
[123]	eval-auc:0.65587
[124]	eval-auc:0.65587
[125]	eval-auc:0.65182
[126]	eval-auc:0.65857
[127]	eval-auc:0.65317
[128]	eval-auc:0.65182
[129]	eval-auc:0.64912
[130]	eval-auc:0.65317
[131]	eval-auc:0.65317
[132]	eval-auc:0.65182
[133]	eval-auc:0.64507
[134]	eval-auc:0.64777
[135]	eval-auc:0.64777
[136]	eval-auc:0.64777
[137]	eval-auc:0.64507
[138]	eval-auc:0.64103
[139]	eval-auc:0.64642
[140]	eval-auc:0.65182
[141]	eval-auc:0.64777
[142]	eval-auc:0.64507
[143]	eval-auc:0.64103
[144]	eval-auc:0.63968
[145]	eval-auc:0.63968
[146]	eval-auc:0.63833
[147]	eval-auc:0.63833
[148]	eval-

...

[106]	eval-auc:0.47090
[107]	eval-auc:0.47354
[108]	eval-auc:0.47619
[109]	eval-auc:0.47619
[110]	eval-auc:0.48148
[111]	eval-auc:0.47619
[112]	eval-auc:0.48148
[113]	eval-auc:0.46825
[114]	eval-auc:0.46825
[115]	eval-auc:0.47619
[116]	eval-auc:0.47354
[117]	eval-auc:0.47354
[118]	eval-auc:0.47354
[119]	eval-auc:0.47354
[120]	eval-auc:0.46825
[121]	eval-auc:0.46825
[122]	eval-auc:0.47354
[123]	eval-auc:0.47619
[124]	eval-auc:0.47090
[125]	eval-auc:0.47619
[126]	eval-auc:0.47884
[127]	eval-auc:0.47884
[128]	eval-auc:0.47354
[129]	eval-auc:0.47619
[130]	eval-auc:0.47619
[131]	eval-auc:0.47619
[132]	eval-auc:0.47619
[133]	eval-auc:0.47619
[134]	eval-auc:0.46825
[135]	eval-auc:0.46296
[136]	eval-auc:0.46561
[137]	eval-auc:0.46825
[138]	eval-auc:0.46825
[139]	eval-auc:0.46825
[140]	eval-auc:0.46296
[141]	eval-auc:0.46825
[142]	eval-auc:0.47090
[143]	eval-auc:0.46825
[144]	eval-auc:0.46825
[145]	eval-auc:0.47090
[146]	eval-auc:0.47090
[147]	eval-auc:0.47090
[148]	eval-auc:0.47090
[149]	eval-

[28]	eval-auc:0.53509
[29]	eval-auc:0.52339
[30]	eval-auc:0.50877
[31]	eval-auc:0.52632
[32]	eval-auc:0.52924
[33]	eval-auc:0.54386
[34]	eval-auc:0.52339
[35]	eval-auc:0.53509
[36]	eval-auc:0.50877
[37]	eval-auc:0.51462
[38]	eval-auc:0.52924
[39]	eval-auc:0.52339
[40]	eval-auc:0.53801
[41]	eval-auc:0.54971
[42]	eval-auc:0.55263
[43]	eval-auc:0.55556
[44]	eval-auc:0.55263
[45]	eval-auc:0.55263
[46]	eval-auc:0.54094
[47]	eval-auc:0.54094
[48]	eval-auc:0.54678
[49]	eval-auc:0.55263
[50]	eval-auc:0.55848
[51]	eval-auc:0.57018
[52]	eval-auc:0.55263
[53]	eval-auc:0.54678
[54]	eval-auc:0.54678
[55]	eval-auc:0.54678
[56]	eval-auc:0.55556
[57]	eval-auc:0.55556
[58]	eval-auc:0.53801
[59]	eval-auc:0.54971
[60]	eval-auc:0.55263
[61]	eval-auc:0.54678
[62]	eval-auc:0.54094
[63]	eval-auc:0.54094
[64]	eval-auc:0.53801
[65]	eval-auc:0.54971
[66]	eval-auc:0.53216
[67]	eval-auc:0.53801
[68]	eval-auc:0.54971
[69]	eval-auc:0.55263
[70]	eval-auc:0.54094
[71]	eval-auc:0.52924
[72]	eval-auc:0.52632
[73]	eval-

[192]	eval-auc:0.49415
[193]	eval-auc:0.49415
[194]	eval-auc:0.49318
[195]	eval-auc:0.49123
[196]	eval-auc:0.49123
[197]	eval-auc:0.49220
[198]	eval-auc:0.49318
[199]	eval-auc:0.49415
[200]	eval-auc:0.49415
0.49220272904483425
[0]	eval-auc:0.38221
[1]	eval-auc:0.36779
[2]	eval-auc:0.38542
[3]	eval-auc:0.40224
[4]	eval-auc:0.36058
[5]	eval-auc:0.37821
[6]	eval-auc:0.38141
[7]	eval-auc:0.40705
[8]	eval-auc:0.38622
[9]	eval-auc:0.39263
[10]	eval-auc:0.39103
[11]	eval-auc:0.36859
[12]	eval-auc:0.36859
[13]	eval-auc:0.36538
[14]	eval-auc:0.36378
[15]	eval-auc:0.36859
[16]	eval-auc:0.37981
[17]	eval-auc:0.37340
[18]	eval-auc:0.38141
[19]	eval-auc:0.38942
[20]	eval-auc:0.37821
[21]	eval-auc:0.38301
[22]	eval-auc:0.38462
[23]	eval-auc:0.38622
[24]	eval-auc:0.38141
[25]	eval-auc:0.39583
[26]	eval-auc:0.39423
[27]	eval-auc:0.40224
[28]	eval-auc:0.39583
[29]	eval-auc:0.39263
[30]	eval-auc:0.39744
[31]	eval-auc:0.38301
[32]	eval-auc:0.38942
[33]	eval-auc:0.38622
[34]	eval-auc:0.38622
[35]	eval-auc

[352]	eval-auc:0.42147
[353]	eval-auc:0.42147
[354]	eval-auc:0.42147
[355]	eval-auc:0.42147
[356]	eval-auc:0.42147
[357]	eval-auc:0.42147
[358]	eval-auc:0.42147
[359]	eval-auc:0.42147
[360]	eval-auc:0.42147
[361]	eval-auc:0.42147
[362]	eval-auc:0.42147
[363]	eval-auc:0.42147
[364]	eval-auc:0.42147
[365]	eval-auc:0.42147
[366]	eval-auc:0.42147
[367]	eval-auc:0.42147
[368]	eval-auc:0.42147
[369]	eval-auc:0.42147
[370]	eval-auc:0.42147
[371]	eval-auc:0.42147
[372]	eval-auc:0.42147
[373]	eval-auc:0.42147
[374]	eval-auc:0.42147
[375]	eval-auc:0.42147
[376]	eval-auc:0.42147
[377]	eval-auc:0.42147
[378]	eval-auc:0.42147
[379]	eval-auc:0.42147
[380]	eval-auc:0.42147
[381]	eval-auc:0.42147
[382]	eval-auc:0.42147
[383]	eval-auc:0.42147
[384]	eval-auc:0.42147
[385]	eval-auc:0.42147
[386]	eval-auc:0.42147
[387]	eval-auc:0.42147
[388]	eval-auc:0.42147
[389]	eval-auc:0.42147
[390]	eval-auc:0.42147
[391]	eval-auc:0.42147
[392]	eval-auc:0.42147
[393]	eval-auc:0.42147
[394]	eval-auc:0.42147
[395]	eval-

[78]	eval-auc:0.37054
[79]	eval-auc:0.37054
[80]	eval-auc:0.36607
[81]	eval-auc:0.36830
[82]	eval-auc:0.37277
[83]	eval-auc:0.37500
[84]	eval-auc:0.37500
[85]	eval-auc:0.37500
[86]	eval-auc:0.37277
[87]	eval-auc:0.37277
[88]	eval-auc:0.37054
[89]	eval-auc:0.37054
[90]	eval-auc:0.37054
[91]	eval-auc:0.37277
[92]	eval-auc:0.37723
[93]	eval-auc:0.37723
[94]	eval-auc:0.38839
[95]	eval-auc:0.38616
[96]	eval-auc:0.38839
[97]	eval-auc:0.39509
[98]	eval-auc:0.39286
[99]	eval-auc:0.39062
[100]	eval-auc:0.39286
[101]	eval-auc:0.39286
[102]	eval-auc:0.39509
[103]	eval-auc:0.39732
[104]	eval-auc:0.39509
[105]	eval-auc:0.39509
[106]	eval-auc:0.38616
[107]	eval-auc:0.38616
[108]	eval-auc:0.39732
[109]	eval-auc:0.38839
[110]	eval-auc:0.39286
[111]	eval-auc:0.39062
[112]	eval-auc:0.39286
[113]	eval-auc:0.39286
[114]	eval-auc:0.39062
[115]	eval-auc:0.39062
[116]	eval-auc:0.39062
[117]	eval-auc:0.39062
[118]	eval-auc:0.38616
[119]	eval-auc:0.38170
[120]	eval-auc:0.37723
[121]	eval-auc:0.37500
[122]	eval

In [21]:
# Average AUC and F1 scores across all groups
avg_auc = np.mean(auc_scores)
avg_f1 = np.mean(f1_scores)

print(f"Avg AUC: {avg_auc}")
print(f"Avg F1 Score: {avg_f1}")

Avg AUC: 0.4914425233244726
Avg F1 Score: 0.4250073473494752
